# Procesado 

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib
import pandas as pd
import datetime

In [9]:
df = pd.read_csv("../../data/processed/train_preprocess_v1.csv")
df_test = pd.read_csv("../../data/processed/test_preprocess_v1.csv")


## Procesado inicial 

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8950 entries, 0 to 8949
Data columns (total 31 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   id                                  8950 non-null   object
 1   label                               8950 non-null   int64 
 2   statement                           8950 non-null   object
 3   subject                             8950 non-null   object
 4   speaker                             8950 non-null   object
 5   speaker_job                         8950 non-null   object
 6   state_info                          8950 non-null   object
 7   party_affiliation                   8950 non-null   object
 8   party_affiliation_uni               8950 non-null   object
 9   party_affiliation_category_map      8950 non-null   object
 10  statement_tokens                    8950 non-null   object
 11  num_tokens                          8950 non-null   int6

Seleccion de categorias a a modelar

In [12]:
X = df['statement']
y = df['label']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)

clf = LinearSVC()
clf.fit(X_train_tfidf, y_train)

X_test_tfidf = vectorizer.transform(X_test)

y_pred = clf.predict(X_test_tfidf)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(classification_report(y_test, y_pred))
print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))



Accuracy: 0.6208530805687204
Classification Report:
              precision    recall  f1-score   support

           0       0.44      0.38      0.40      1014
           1       0.70      0.75      0.72      1940

    accuracy                           0.62      2954
   macro avg       0.57      0.56      0.56      2954
weighted avg       0.61      0.62      0.61      2954

Confusion Matrix:
[[ 381  633]
 [ 487 1453]]


In [14]:
X_test_raw = df_test['statement']
X_test_counts = count_vect.transform(X_test_raw)
X_test_tfidf   = tfidf_transformer.transform(X_test_counts)

y_pred = clf.predict(X_test_tfidf)

#  Crear el DataFrame de submission
submission = pd.DataFrame({
    'id':    df_test['id'],
    'label': y_pred
})

#  Guardar a CSV con fecha en el nombre
filename = f"svc_submission_{datetime.datetime.now().strftime('%Y%m%d')}.csv"
submission.to_csv(filename, index=False, columns=['id', 'label'])

print(f"✅ Submission generada correctamente: '{filename}'")
print(submission.head())

✅ Submission generada correctamente: 'svc_submission_20250513.csv'
            id  label
0  dc32e5ffa8b      0
1  aa49bb41cab      1
2  dddc8d12ac1      1
3  bcfe8f51667      1
4  eedbbaff5ab      1
